In [1]:
from pymongo import MongoClient
import matplotlib.pyplot as plt
import folium
import numpy as np

def docs_from_mongodb_collection(db, coll_name):
    coll = db[coll_name]
    docs = []
    for doc in coll.find():
        docs.append(doc)
    return docs

class StravaActivity(object):
    ''' a class for attributes of a Strava activity '''

    def __init__(self, d):
        self.name = d['name']
        self.type = None
        self.dist = d['distance']
        self.time = d['ts_id']
        self.elev = d['altitude']
        self.lat = d['lat']
        self.lon = d['lon']
        self.tot_distance = self.dist[-1]
        self.tot_time = self.time[-1]
        self.start_elev = self.elev[0]
        self.slopes = []
        self.velocities = []
        self.mph = []

    def calc_slopes(self):
        ''' calculate slopes for each interval'''
        # pad the end with ~ last val
        xx = self.dist + [self.dist[-1] + 0.1] # plus dx to avoid div/0
        yy = self.elev + [self.elev[-1]]
        for idx in range(len(xx) - 1):
            self.slopes.append((yy[idx + 1] - yy[idx])/(xx[idx + 1] - xx[idx]))

    def calc_velocities(self):
        ''' calculate velocities for each interval'''
        xx = self.dist + [self.dist[-1] + 0.1] # plus dx to avoid div/0
        tt = self.time + [self.time[-1] + 0.1]
        for idx in range(len(xx) - 1):
            self.velocities.append((xx[idx + 1] - xx[idx])/(tt[idx + 1] - tt[idx]))

    def standard_units(self):
        ''' calculate standard units for easy interpretation'''
        meter_mile_conv = 1600
        # migrate numpy arrays up for derivative
        vel = np.array(self.velocities)
        self.mph = vel * 1/meter_mile_conv * 60 * 60

    def fit(self):
        ''' perform all calculations '''
        self.calc_slopes()
        self.calc_velocities()
        self.standard_units()

    def plot_xy(self):
        plt.plot(self.lon, self.lat, 'ok')
        plt.axis('equal')
        plt.xlabel('longitude')
        plt.ylabel('latitude')
        plt.title(self.name + ' map')
        plt.savefig('../figs/xy/{}_xy.png'.format(self.name), dpi=250)
        plt.close()

    def plot_map_folium(self):
        m = folium.Map(location=[self.lat, self.lon],
              tiles='OpenStreetMap')
        return m

    def plot_elev_profiles(self):
        plt.plot(self.dist, self.elev, 'ok', markersize=2)
        plt.xlabel('distance (m)')
        plt.ylabel('elevation (m)')
        plt.title(self.name + ' elevation profile')
        #plt.axis('equal')
        #plt.show()
        plt.savefig('../figs/elev_profiles/{}_elev.png'.format(self.name), dpi=250)
        plt.close()

    def plot_slopes(self):
        plt.plot(self.dist, self.slopes, 'og', markersize=2)
        plt.xlabel('distance (m)')
        plt.ylabel('slope')
        plt.title(self.name + ' slope')
        plt.savefig('../figs/slope_profiles/{}_slope.png'.format(self.name), dpi=250)
        plt.close()

    def plot_mph(self):
        plt.plot(self.dist, self.mph, 'ob', markersize=2)
        plt.xlabel('distance (m)')
        plt.ylabel('mph')
        plt.title(self.name + ' speed')
        plt.savefig('../figs/speed/{}_speed.png'.format(self.name), dpi=250)
        plt.close()

    def plot_all(self):
        ''' generate all plots '''
        self.plot_xy()
        self.plot_elev_profiles()
        self.plot_slopes()
        self.plot_mph()

In [2]:
# initiate mongo database and connect
client = MongoClient('localhost',27017)  # Establish connection to persistent storage
db = client.Strava  # Access/Initiate Database
acts_list = docs_from_mongodb_collection(db,'Drew_Thayer_user') # list of activities, with id

# list of activities
activities = [list(item.values())[1] for item in acts_list]

In [24]:
# plot all activities in map together
latlon_all = []
for activity in activities:
   act = StravaActivity(activity)
   act.fit()
   lat = act.lat
   lon = act.lon
   latlon_all.append((lat,lon))

In [25]:
len(latlon_all)


10

In [18]:
# add to folium map
from folium.plugins import HeatMap
import pandas as pd

latlon_1 = latlon_all[0]

heat_df = pd.DataFrame({'lat': latlon_1[0], 'lon':latlon_1[1]})

m = folium.Map(location=[latlon_1[0][0], latlon_1[1][0]], zoom_start=20)

heat_data = [[row['lat'], row['lon']] for index, row in heat_df.iterrows()]
HeatMap(heat_data).add_to(m)

folium.TileLayer('stamenterrain').add_to(m)

In [19]:
m

## add all rides to folium map

In [26]:
from folium.plugins import HeatMap
import pandas as pd

m = folium.Map(location=[latlon_all[0][0][0], latlon_all[0][1][0]], zoom_start=20)
folium.TileLayer('stamenterrain').add_to(m)

for latlon in latlon_all:
    heat_df = pd.DataFrame({'lat': latlon[0], 'lon':latlon[1]})

    heat_data = [[row['lat'], row['lon']] for index, row in heat_df.iterrows()]
    HeatMap(heat_data).add_to(m)

    

In [27]:
m